# HQIV Perturbation Solver Demo

This notebook demonstrates the Horizon-Quantized Informational Vacuum (HQIV) perturbation solver.

**Paper Reference:** "Horizon-Quantized Informational Vacuum (HQIV): A Covariant Baryon-Only Cosmological Framework from Quantised Inertia"

## Key Features:
- Background cosmology with horizon term
- Scalar perturbations with modified inertia
- Vector (vorticity) perturbations with horizon coupling
- CMB angular power spectrum $C_\ell^{TT}$

In [ ]:
# Import the solver
import sys
sys.path.insert(0, '..')

from hqiv_solver import HQIVPerturbations, CosmologicalBackground
import numpy as np
import matplotlib.pyplot as plt

print("HQIV Perturbation Solver Demo")
print("=" * 50)

## 1. Background Cosmology Comparison

Compare HQIV and $\Lambda$CDM background evolution.

In [ ]:
# Create both cosmologies
hqiv_bg = CosmologicalBackground(
    H0=73.2,           # km/s/Mpc (SH0ES value)
    Om_m=0.031,        # Baryon density only
    hqiv_on=True,
    beta=1.02,
    Om_h=1.00,
    n_h=1.04
)

lcdm_bg = CosmologicalBackground(
    H0=67.4,           # km/s/Mpc (Planck value)
    Om_m=0.315,        # Total matter density
    hqiv_on=False
)

# Compute backgrounds
hqiv_bg.compute_background()
lcdm_bg.compute_background()

# Print summaries
print("\n" + "="*60)
print("HQIV Background:")
hqiv_bg.summary()

print("\n" + "="*60)
print("ΛCDM Background:")
lcdm_bg.summary()

In [ ]:
# Plot background comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

a = hqiv_bg._a_arr

# H(a) comparison
ax = axes[0, 0]
ax.loglog(a, hqiv_bg.H_km(a), 'b-', label='HQIV', linewidth=2)
ax.loglog(a, lcdm_bg.H_km(a), 'r--', label='ΛCDM', linewidth=2)
ax.set_xlabel('Scale factor a')
ax.set_ylabel('H(a) [km/s/Mpc]')
ax.set_title('Hubble Parameter Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

# G_eff(a) for HQIV
ax = axes[0, 1]
ax.semilogx(a, hqiv_bg.G_eff(a), 'b-', linewidth=2)
ax.axhline(1.0, color='k', linestyle='--', alpha=0.5)
ax.set_xlabel('Scale factor a')
ax.set_ylabel('$G_{eff}/G_0$')
ax.set_title('Effective Gravitational Coupling (HQIV)')
ax.grid(True, alpha=0.3)

# β(a) evolution
ax = axes[1, 0]
ax.semilogx(a, hqiv_bg.beta_a(a), 'b-', linewidth=2)
ax.axhline(1.0, color='k', linestyle='--', alpha=0.5)
ax.set_xlabel('Scale factor a')
ax.set_ylabel('β(a)')
ax.set_title('Horizon-Smoothing Parameter Evolution')
ax.grid(True, alpha=0.3)

# Proper time comparison
ax = axes[1, 1]
ax.semilogx(a, hqiv_bg._t_arr / 3.1536e16, 'b-', label='HQIV', linewidth=2)
ax.semilogx(a, lcdm_bg._t_arr / 3.1536e16, 'r--', label='ΛCDM', linewidth=2)
ax.set_xlabel('Scale factor a')
ax.set_ylabel('Proper time t [Gyr]')
ax.set_title('Cosmic Time Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('background_comparison.png', dpi=150)
plt.show()
print("\nSaved background_comparison.png")

## 2. Scalar Perturbations

Evolve density contrast $\delta$, velocity divergence $\theta$, and potential $\Phi$.

In [ ]:
# Run the full HQIV solver
print("Running HQIV solver...")

solver = HQIVPerturbations(
    H0=73.2,
    Om_m=0.031,
    hqiv_on=True,
    beta=1.02,
    Om_h=1.00,
    n_h=1.04
)

# Use fewer k modes for speed in this demo
k_array = np.logspace(-2, 0, 30)
results = solver.run(k_array=k_array, n_pts=300, verbose=True)

In [ ]:
# Plot scalar perturbation evolution
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Growth factor
ax = axes[0, 0]
a_D = solver.scalar_results['growth_factor']['a']
D = solver.scalar_results['growth_factor']['D']
ax.semilogx(a_D, D, 'b-', linewidth=2)
ax.set_xlabel('Scale factor a')
ax.set_ylabel('D(a)')
ax.set_title('Linear Growth Factor (HQIV)')
ax.grid(True, alpha=0.3)

# Transfer function
ax = axes[0, 1]
k = solver.scalar_results['k_array']
delta = solver.scalar_results['delta_final']
T = delta / np.max(np.abs(delta))
ax.semilogx(k, T, 'b-', linewidth=2)
ax.set_xlabel('k [Mpc⁻¹]')
ax.set_ylabel('T(k)')
ax.set_title('Transfer Function')
ax.grid(True, alpha=0.3)

# Density evolution for a specific k
ax = axes[1, 0]
if 'k_0.100' in solver.scalar_results:
    res = solver.scalar_results['k_0.100']
    ax.loglog(res['a'], np.abs(res['delta']), 'b-', label='|δ|', linewidth=2)
    ax.loglog(res['a'], np.abs(res['Phi']), 'r--', label='|Φ|', linewidth=2)
    ax.set_xlabel('Scale factor a')
    ax.set_ylabel('Perturbation amplitude')
    ax.set_title('Evolution for k = 0.1 Mpc⁻¹')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Potential evolution
ax = axes[1, 1]
if 'k_0.100' in solver.scalar_results:
    res = solver.scalar_results['k_0.100']
    ax.semilogx(res['a'], res['Phi'], 'b-', linewidth=2)
    ax.set_xlabel('Scale factor a')
    ax.set_ylabel('Φ')
    ax.set_title('Newtonian Potential (k = 0.1 Mpc⁻¹)')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('scalar_perturbations.png', dpi=150)
plt.show()
print("\nSaved scalar_perturbations.png")

## 3. Vorticity Evolution

Paper Eq. 12: $\partial\omega/\partial t + (\mathbf{v}\cdot\nabla)\omega = \beta H (\omega \cdot \hat{e}_\Theta)$

In standard cosmology, vorticity decays as $\omega \propto a^{-2}$. In HQIV, the horizon coupling can amplify it.

In [ ]:
# Plot vorticity evolution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

if solver.vector_results:
    a_v = solver.vector_results['a']
    A_omega = solver.vector_results['amplification']
    
    # Vorticity magnitude
    ax = axes[0]
    ax.loglog(a_v, A_omega, 'b-', label='HQIV', linewidth=2)
    ax.loglog(a_v, (a_v / a_v[0])**(-2), 'r--', label='Standard: a⁻²', linewidth=1.5)
    ax.set_xlabel('Scale factor a')
    ax.set_ylabel('|ω| / |ω₀|')
    ax.set_title('Vorticity Evolution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim([1e-10, 1e10])
    
    # Vorticity components
    ax = axes[1]
    omega = solver.vector_results['omega']
    ax.loglog(a_v, np.abs(omega[:, 0]), 'r-', label='ω_x', linewidth=2)
    ax.loglog(a_v, np.abs(omega[:, 1]), 'g-', label='ω_y', linewidth=2)
    ax.loglog(a_v, np.abs(omega[:, 2]), 'b-', label='ω_z (parallel to ê_Θ)', linewidth=2)
    ax.set_xlabel('Scale factor a')
    ax.set_ylabel('|ω_i|')
    ax.set_title('Vorticity Components (HQIV)')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('vorticity_evolution.png', dpi=150)
plt.show()
print("\nSaved vorticity_evolution.png")

print(f"\nVorticity growth exponent: {solver.vector_results['beta_eff']:.4f}")
print("(Standard cosmology: β = -2)")

## 4. CMB Power Spectrum

Compute $C_\ell^{TT}$ and find the first acoustic peak position.

In [ ]:
# Plot CMB spectrum
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

if solver.observables:
    ell = solver.observables['ell']
    Dl = solver.observables['Dl_TT']
    ell_peak = solver.observables['ell_peak']
    
    # CMB spectrum
    ax = axes[0]
    ax.plot(ell, Dl, 'b-', linewidth=2)
    ax.axvline(ell_peak, color='r', linestyle=':', alpha=0.7, 
              label=f'Peak: ℓ = {ell_peak:.0f}')
    ax.set_xlabel('Multipole ℓ')
    ax.set_ylabel('$D_\ell^{TT}$ [μK²]')
    ax.set_title('CMB Temperature Power Spectrum (HQIV)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim([2, 500])
    
    # Key scales
    ax = axes[1]
    r_s = solver.observables['sound_horizon']
    eta_0 = solver.observables['conformal_time']
    
    labels = ['Sound horizon\nr_s [Mpc]', 'Conformal time\nη₀ [Mpc]', 'Peak position\nℓ = πη₀/r_s']
    values = [r_s, eta_0, np.pi * eta_0 / r_s]
    
    bars = ax.bar(labels, values, color=['blue', 'green', 'red'], alpha=0.7)
    ax.set_ylabel('Value')
    ax.set_title('Key CMB Scales')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(values),
               f'{val:.1f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('cmb_spectrum.png', dpi=150)
plt.show()
print("\nSaved cmb_spectrum.png")

print(f"\nFirst acoustic peak: ℓ = {solver.observables['ell_peak']:.0f}")
print("Standard ΛCDM: ℓ ~ 220-280")

## 5. Comparison with ΛCDM

Compare growth suppression and CMB peak shift.

In [ ]:
# Compare with ΛCDM
print("Comparing HQIV with ΛCDM...")
comparison = solver.compare_with_lcdm(k_array=k_array, n_pts=300, verbose=True)

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Growth factor comparison
ax = axes[0, 0]
a_c = comparison['a']
ax.semilogx(a_c, comparison['D_HQIV'], 'b-', label='HQIV', linewidth=2)
ax.semilogx(a_c, comparison['D_LCDM'], 'r--', label='ΛCDM', linewidth=2)
ax.set_xlabel('Scale factor a')
ax.set_ylabel('D(a)')
ax.set_title('Linear Growth Factor Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

# Growth suppression
ax = axes[0, 1]
supp = comparison['growth_suppression']
ax.semilogx(a_c, supp, 'b-', linewidth=2)
ax.axhline(1.0, color='k', linestyle='--', alpha=0.5)
ax.axhline(0.36, color='g', linestyle=':', alpha=0.7, label='Paper prediction (0.36)')
ax.set_xlabel('Scale factor a')
ax.set_ylabel('$D_{HQIV}/D_{ΛCDM}$')
ax.set_title('Growth Suppression Factor')
ax.legend()
ax.grid(True, alpha=0.3)

# CMB comparison
ax = axes[1, 0]
hqiv_obs = comparison['hqiv_results']['observables']
lcdm_obs = comparison['lcdm_results']['observables']

ax.plot(hqiv_obs['ell'], hqiv_obs['Dl_TT'], 'b-', label='HQIV', linewidth=2)
ax.plot(lcdm_obs['ell'], lcdm_obs['Dl_TT'], 'r--', label='ΛCDM', linewidth=2)
ax.axvline(hqiv_obs['ell_peak'], color='b', linestyle=':', alpha=0.5)
ax.axvline(lcdm_obs['ell_peak'], color='r', linestyle=':', alpha=0.5)
ax.set_xlabel('Multipole ℓ')
ax.set_ylabel('$D_\ell^{TT}$ [μK²]')
ax.set_title('CMB Spectrum Comparison')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([2, 500])

# Summary text
ax = axes[1, 1]
ax.axis('off')

summary = f"""
Comparison Summary
{'='*40}

Growth Suppression:
  At a=0.5: {np.interp(0.5, a_c, supp):.4f}
  Paper prediction: ~0.36

CMB Peak Position:
  HQIV: ℓ = {hqiv_obs['ell_peak']:.0f}
  ΛCDM: ℓ = {lcdm_obs['ell_peak']:.0f}
  Shift: Δℓ = {comparison['peak_shift']:.0f}

Vorticity:
  HQIV exponent: {comparison['hqiv_results']['vectors']['beta_eff']:.2f}
  Standard: β = -2

Background:
  HQIV age: {comparison['hqiv_results']['background'].age_today():.1f} Gyr
  ΛCDM age: {comparison['lcdm_results']['background'].age_today():.1f} Gyr
"""

ax.text(0.1, 0.5, summary, transform=ax.transAxes, fontsize=11,
        verticalalignment='center', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('comparison.png', dpi=150)
plt.show()
print("\nSaved comparison.png")

## 6. Summary

Print final summary of all results.

In [ ]:
# Print final summary
solver.summary()

## Key Results

This demo shows:

1. **Background**: HQIV gives an older universe (~17 Gyr) compared to ΛCDM (~13.8 Gyr), addressing JWST early-galaxy timing.

2. **Growth Suppression**: The linear growth factor is suppressed by ~0.36× relative to ΛCDM, a falsifiable prediction.

3. **Vorticity**: Unlike standard cosmology where vorticity decays as $a^{-2}$, HQIV can amplify or sustain vorticity through the horizon coupling term.

4. **CMB Peak**: The first acoustic peak position may shift due to modified sound horizon and angular diameter distance.

### Next Steps

- Implement full photon-baryon fluid evolution for accurate CMB
- Add tensor modes for B-mode polarization
- Couple to N-body simulations for structure formation
- Compare with Planck 2018 data